- How much energy is to purchase/sell in different time periods subject to maximize profit or minimize risk 
- Based on forecasts, contstraints, and market prices
- For battery, constraints are battery capacity, charge/discharge rates

**Objective**
**Maximize Profit = revenue - cost**

Revenues are determined from models. Cost, probably from background information.

✅ Example: Battery Arbitrage over 3 Time Slots
You have a battery that can buy/sell electricity based on day-ahead prices:

- **Prices:** Electricity pricing (£/MWh) for three time periods [£50, £100, £60]. Price is the same for charging or discharing, it can be considered positive for discharging while negative for charging.
- **Revenue and cost**: Discharging the battery generates revenue while charging the battery is a cost for asset. Obviously, you can't keep discharging battery, it has to be charged to discharge.

- **Constraints:** Max charge/discharge: 10 MWh per hour, total capacity: 10 MWh, Start and end state of charge (SoC): 0

In [ ]:
# Battery parameters
storage_capacity = 20 # Maximum amount of storing energy (MWh)
print('Battery storage capacity:', storage_capacity, 'MWh')
power_rating = 10 # maximum amount of charge to be relased in an hour (MW)
print('The maximum rate of battery discharge:', power_rating, 'MW')
print('The battery can run at full capacity for', str(storage_capacity)+str('MWh/')+str(power_rating)+str('MW') )
discharging_eff = 0.9 
print('Maximum discharge would be:', storage_capacity*discharging_eff, 'MWh')
charging_eff = 0.9 
print('To charge maximum capacity of', storage_capacity, 'MWh, required energy would be:', np.round(storage_capacity/charging_eff,2), 'MWh')

Battery storage capacity: 20 MWh
The maximum rate of battery discharge: 10 MW
The battery can run at full capacity for 20MWh/10MW
Maximum discharge would be: 18.0 MWh
To charge maximum capacity of 20 MWh, required energy would be: 22.22 MWh


In [15]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

**Objective function**: $\min_x {c}^T x$ 

- $A_{ub} x \leq b_{ub}$
- $A_{eq} x = b_{eq}$
- $l \leq x \leq u$

- c: 1-D array, coefficients of the linear objective fucntion to be minimized
- x: vector of decision variables, 
- $A_{ub}$ and $A_{eq}$ are upper bounds and equality metrics.
- $b_{ub}$ and $b_{eq}$ are inequality constraint vectors.
- bounds: sequence of (min, max) pairs for each element in x, defining the minimum and maximum values of that decision variable. If a single tuple (min, max) is provided, then min and max will serve as bounds for all decision variables.  


In [ ]:
prices = np.array([50, 100, 60])  
# Objective Maximize profit = maximize(price*discharge - price*charge)
# Objective functions coefficients - linprog minimizes, so we flip the sig
obj_coeff = -1*np.concatenate([prices, -prices]) #discharge, charge 
obj_coeff

array([ -50, -100,  -60,   50,  100,   60])

In [83]:
# Bounds sequence of (min, max) pairs for each element in x,
bounds = [(0, 100)]*6
bounds

[(0, 100), (0, 100), (0, 100), (0, 100), (0, 100), (0, 100)]

**Constraints: SOC**

$\text{SOC}(t) = \sum_{i =1}^t {\text{charge}_{i}} - \sum_{i=1}^t {\text{discharge}_i}$

$\text{SOC}(1) \leq 10 ⇒ -{d_1} + c_1 \leq 10$ ⇒ [-1 0 0 1 0 0]

$\text{SOC}(2) \leq 10 ⇒ -{d_1} + c_1 -{d_2} + c_2\leq 10$ ⇒ [-1 -1 0 1 1 0]

$\text{SOC}(3) \leq 10 ⇒ -{d_1} + c_1 -{d_2} + c_2 -{d_3} + c_3\leq 10$ ⇒ [-1 -1 -1 1 1 1]

In [95]:
# Ax <= b, soc_upper 
A_ub = []
b_ub = []
for t in range(3): # t1, t2, t3
    soc_upper = [0]*6 # Initial variables - 0, modified for SOC upper limit
    soc_lower = [0]*6 # Initial variables - 0, modified for SOC lower limit
    dis_index = np.arange(0, t+1, 1)
    charge_index = np.arange(3, t+4, 1)
    for index in dis_index:
        soc_upper[index] = -1
        soc_lower[index] = 1
    for index in charge_index:
        soc_upper[index] = 0.5
        soc_lower[index] = -0.5
    A_ub.append(soc_upper)
    b_ub.append(100)
    A_ub.append(soc_lower)
    b_ub.append(0)

In [94]:
A_ub

[[-1, 0, 0, 1, 0, 0],
 [0.5, 0, 0, -0.5, 0, 0],
 [-1, -1, 0, 1, 1, 0],
 [0.5, 0.5, 0, -0.5, -0.5, 0],
 [-1, -1, -1, 1, 1, 1],
 [0.5, 0.5, 0.5, -0.5, -0.5, -0.5]]

In [96]:
b_ub, c

([100, 0, 100, 0, 100, 0], array([ -50, -100,  -60,   50,  100,   60]))

In [97]:

res = linprog(c = obj_coeff, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")
res.x

array([  0.,  50.,   0., 100.,   0.,   0.])

In [86]:
res

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -500.0
              x: [ 0.000e+00  1.000e+01  0.000e+00  1.000e+01  0.000e+00
                  -0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  1.000e+01  0.000e+00  1.000e+01
                              0.000e+00 -0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [ 1.000e+02  9.000e+01  1.000e+02  9.000e+01
                              1.000e+02  1.000e+02]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  1.000e+01  1.000e+01  0.000e+00
                              1.000e+01  0.000e+00]
                 marginals: [-5.000e+01 -0.00

In [85]:
res.x

array([ 0., 10.,  0., 10.,  0., -0.])

In [ ]:
from scipy.optimize import linprog
import numpy as np

# Market prices (£/MWh)
prices = np.array([50, 100, 60])  

# Decision variables: charge/discharge at each hour (positive = discharge, negative = charge)
# We'll use positive variables: discharge (x1), charge (x2), SoC variables

# Variables: [d1, d2, d3, c1, c2, c3]
# Objective: Maximize profit → Maximize (price ⋅ discharge) - (price ⋅ charge)
# linprog minimizes, so we flip the sign
c = -1 * np.concatenate([prices, -prices])  # [discharge, charge]

# Bounds for each variable
bounds = [(0, 10)] * 6  # max 10 MWh discharge/charge per hour

# Constraints
A = []
b = []

# 1. SoC constraint (cumulative energy): SoC = ∑(charge - discharge)
# SoC at each step should be within [0, 10]

# Build SoC at t1, t2, t3 (SoC must be <= capacity)
for t in range(3):
    row = [0]*6
    for i in range(t+1):
        row[i+3] = 1   # charge
        row[i] = -1    # discharge
    A.append(row)
    b.append(10)  # max SoC

# SoC >= 0 → -SoC <= 0
for t in range(3):
    row = [0]*6
    for i in range(t+1):
        row[i+3] = -1   # charge
        row[i] = 1      # discharge
    A.append(row)
    b.append(0)  # min SoC

